In [2]:
import pandas as pd
import requests

provinces = {
    "Hà Nội": (21.0285, 105.8542),
    "Hồ Chí Minh": (10.762622, 106.660172),
    "Đà Nẵng": (16.047079, 108.206230),
    "Hải Phòng": (20.844912, 106.688085),
    "Cần Thơ": (10.045162, 105.746857)
}

# event_dates = ["2025-05-01", "2025-05-02"]
start_date = "2025-05-08"
today = pd.to_datetime("today").date()
event_dates = pd.date_range(start=start_date, end=today).strftime('%Y-%m-%d').tolist()

results = []

for date in event_dates:
    for province, (lat, lon) in provinces.items():
        url = (
            f"https://api.open-meteo.com/v1/forecast?"
            f"latitude={lat}&longitude={lon}&start_date={date}&end_date={date}"
            f"&daily=temperature_2m_max,temperature_2m_min,windspeed_10m_max,weathercode"
            f"&timezone=auto"
        )
        r = requests.get(url)
        if r.ok:
            data = r.json().get("daily", {})
            if data:
                results.append({
                    "province": province,
                    "date": date,
                    "temp_max": data["temperature_2m_max"][0],
                    "temp_min": data["temperature_2m_min"][0],
                    "windspeed_max": data["windspeed_10m_max"][0],
                    "weathercode": data["weathercode"][0]
                })

df = pd.DataFrame(results)
print(df)



       province        date  temp_max  temp_min  windspeed_max  weathercode
0        Hà Nội  2025-05-10      30.6      25.4           17.9           96
1   Hồ Chí Minh  2025-05-10      32.2      27.0            7.3           95
2       Đà Nẵng  2025-05-10      31.2      27.5           14.3           80
3     Hải Phòng  2025-05-10      29.9      26.4           12.6           96
4       Cần Thơ  2025-05-10      33.1      26.8           11.1           95
5        Hà Nội  2025-05-11      27.1      20.4           15.5           81
6   Hồ Chí Minh  2025-05-11      32.7      26.3            5.8            3
7       Đà Nẵng  2025-05-11      29.8      26.2           18.5           95
8     Hải Phòng  2025-05-11      26.6      20.4           17.1           80
9       Cần Thơ  2025-05-11      32.8      25.7            8.0           95
10       Hà Nội  2025-05-12      29.0      20.9            8.9            3
11  Hồ Chí Minh  2025-05-12      32.7      26.2           11.4           95
12      Đà N

In [5]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

project_id = "fsds-bi"
dataset_id = "sample_datawarehouse"
table_id = "fake_data_daily"
key_path = "/Users/nguyentienhuy/Documents/fsds/BI/material/Lesson 15 - Python 3/fsds-bi-96a792b2baf8.json"

# Bước 3: Tạo client BigQuery
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=project_id)

# Bước 4: Tạo tham chiếu bảng
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Bước 5: Cấu hình ghi đè bảng nếu đã tồn tại
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    autodetect=True,  
)

# Bước 6: Load DataFrame vào BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  

# Bước 7: Xác nhận
print(f"✅ Đã tải {job.output_rows} dòng vào bảng {table_ref}")

/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✅ Đã tải 15 dòng vào bảng fsds-bi.sample_datawarehouse.fake_data_daily
